In [6]:
import pandas as pd
import sqlalchemy as sqla

In [7]:
query = """
    SELECT
	SalesKey
	,YEAR(DateKey) AS SaleYear
	,DATENAME(MONTH, DateKey) AS SaleMonth
	,DAY(DateKey) as SaleDay
	,CAST(DateKey AS DATE) AS SaleDate
	,e.ChannelName
	,c.SalesTerritoryCountry
	,c.SalesTerritoryRegion
	,c.SalesTerritoryName
	,d.CityName
	,b.StoreType
	,b.StoreName
	,h.ProductCategoryName
	,g.ProductSubcategoryName
	,f.ClassName
	,f.BrandName
	,f.ProductName
	,TotalCost
	,SalesQuantity
	,SalesAmount
	,DiscountQuantity
	,DiscountAmount
	,ReturnQuantity
	,ReturnAmount	
FROM
	FactSales a
LEFT JOIN DimStore b
	ON a.StoreKey = b.StoreKey
LEFT JOIN DimSalesTerritory c
	ON b.GeographyKey = c.GeographyKey
LEFT JOIN DimGeography d
	ON b.GeographyKey = d.GeographyKey
LEFT JOIN DimChannel e
	ON a.channelKey = e.ChannelKey
LEFT JOIN DimProduct f
	ON a.ProductKey = f.ProductKey
LEFT JOIN DimProductSubcategory g
	ON f.ProductSubcategoryKey = g.ProductSubcategoryKey
LEFT JOIN DimProductCategory h
	ON g.ProductCategoryKey = h.ProductCategoryKey
    """

In [16]:
servidor = 'localhost'
banco_de_dados = 'ContosoRetailDW'
driver = 'ODBC Driver 17 for SQL Server'

conexao_str = f'mssql+pyodbc://{servidor}/{banco_de_dados}?driver={driver}&trusted_connection=yes'

engine = sqla.create_engine(conexao_str)

In [17]:
dados = pd.read_sql_query(query, engine)

In [20]:
dados.head()

,SalesKey,SaleYear,SaleMonth,SaleDay,SaleDate,ChannelName,SalesTerritoryCountry,SalesTerritoryRegion,SalesTerritoryName,CityName,...,ClassName,BrandName,ProductName,TotalCost,SalesQuantity,SalesAmount,DiscountQuantity,DiscountAmount,ReturnQuantity,ReturnAmount
0,1,2007,Janeiro,2,2007-01-02,Store,UK,England,Baildon,Baildon,...,Regular,A. Datum,A. Datum Point Shoot Digital Camera M500 Black,728.40,8,1544.400,1,39.600,0,0.0
1,2,2007,Fevereiro,12,2007-02-12,Reseller,United States,Washington,Seattle,Seattle,...,Economy,Contoso,Contoso Battery charger - bike E200 Black,40.60,4,78.605,1,0.995,0,0.0
2,3,2008,Janeiro,24,2008-01-24,Store,United States,Massachusetts,Cambridge,Cambridge,...,Regular,Fabrikam,Fabrikam Budget Moviemaker 2/3'' 17mm E100 White,1881.27,9,3628.500,3,61.500,0,0.0
3,4,2008,Janeiro,13,2008-01-13,Online,Germany,Berlin,Berlin,Berlin,...,Regular,The Phone Company,The Phone Company Touch Screen Phones 4-Wire/O...,1063.20,8,2254.200,1,57.800,0,0.0
4,5,2008,Janeiro,22,2008-01-22,Online,Germany,Berlin,Berlin,Berlin,...,Regular,Fabrikam,"Fabrikam SLR Camera 35"" X358 Blue",3468.48,24,10207.080,3,261.720,0,0.0
